(sbd_cmdstanpy)=
# Simulated Bayes Calibration with CmdStanPy

Uses toy model from {ref}`pystan_schema`.

In [1]:
from cmdstanpy import CmdStanModel, write_stan_json

import arviz as az
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# read data
data = pd.read_csv("../../schema/linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
github_commits = data.commits.values
names = data.index.values
N = len(names)

In [3]:
prior_model = CmdStanModel(stan_file="linreg_prior.stan")

In [4]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
write_stan_json("linreg_prior.data.json", linreg_prior_data_dict)
prior_fit = prior_model.sample(data="linreg_prior.data.json", chains=2, iter_sampling=100)

03:19:07 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

03:19:07 - cmdstanpy - INFO - CmdStan done processing.


In [5]:
idata_kwargs = dict(
    prior_predictive=["slack_comments_hat", "github_commits_hat"],
    posterior_predictive=["slack_comments_hat", "github_commits_hat"],
    log_likelihood={
        "slack_comments": "log_likelihood_slack_comments",
        "github_commits": "log_likelihood_github_commits",
    },
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "github_commits": ["developer"],
        "slack_comments_hat": ["developer"],
        "github_commits_hat": ["developer"],
        "time_since_joined": ["developer"],
    },
)
idata_orig = az.from_cmdstanpy(prior=prior_fit, **idata_kwargs).stack(
    prior_draw=["chain", "draw"], groups="prior_groups"
)
idata_orig

Inference data with groups:
	> prior
	> prior_predictive
	> sample_stats_prior

In [6]:
model = CmdStanModel(stan_file="linreg_model.stan")

In [7]:
linreg_base_data = {
    "N": N,
    "time_since_joined": time_since_joined,
}

In [8]:
prior_pred = idata_orig.prior_predictive
S = prior_pred.dims["prior_draw"]
idata_list = []
for i in range(S):
    data_s = prior_pred.isel(prior_draw=i)
    data_dict = {
        "slack_comments": data_s["slack_comments_hat"].values,
        "github_commits": data_s["github_commits_hat"].values,
        **linreg_base_data,
    }
    write_stan_json("linreg_data.json", data_dict)
    fit = model.sample(data="linreg_data.json")
    idata = az.from_cmdstanpy(posterior=fit, **idata_kwargs)
    idata_list.append(idata)

03:19:08 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

...

chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

03:20:45 - cmdstanpy - INFO - CmdStan done processing.


In [9]:
post = xr.concat((idata.posterior for idata in idata_list), dim="prior_draws")
post_pred = xr.concat((idata.posterior_predictive for idata in idata_list), dim="prior_draws")

idata_orig.add_groups(posterior=post, posterior_preditive=post_pred)

/home/oriol/Public/arviz/arviz/data/inference_data.py:1427: UserWarning: The group posterior_preditive is not defined in the InferenceData scheme
  warnings.warn(


In [10]:
idata_orig

Inference data with groups:
	> posterior
	> prior
	> prior_predictive
	> sample_stats_prior
	> posterior_preditive

The prior sampling code is wrong because it doesn't take the positive constraint into account, so to get things working quickly I removed the constraint in the data block. I think it is good enough for the proof of concept here. 